<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

In [1]:
# This cell is for papermill parameters. DO NOT CHANGE THE VARIABLE NAMES.
# Default values for manual execution (papermill will override these)
input_geojson_path = None
output_csv_path = None

In [2]:
# Parameters
input_geojson_path = "training-data/postcards/nm-mana-south-26072025_postcard.geojson"
output_csv_path = "automated_notebook_runs/sampled_data_csvs/nm-mana-south-26072025_postcard_sampled_data.csv"


## Step 1.1: Configure the environment

In [3]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [4]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [5]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "serua_2"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-serua_2-27072025


## Step 1.2: Configure STAC access and search parameters

In [6]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [7]:
training = gpd.read_file(input_geojson_path)
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

In [8]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [9]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 1MB
Dimensions:      (y: 153, x: 222, time: 1)
Coordinates:
  * y            (y) float64 1kB -1.987e+06 -1.987e+06 ... -1.988e+06 -1.988e+06
  * x            (x) float64 2kB 3.017e+06 3.017e+06 ... 3.019e+06 3.019e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 68kB dask.array<chunksize=(1, 153, 222), meta=np.ndarray>
    red          (time, y, x) uint16 68kB dask.array<chunksize=(1, 153, 222), meta=np.ndarray>
    blue         (time, y, x) uint16 68kB dask.array<chunksize=(1, 153, 222), meta=np.ndarray>
    green        (time, y, x) uint16 68kB dask.array<chunksize=(1, 153, 222), meta=np.ndarray>
    emad         (time, y, x) float32 136kB dask.array<chunksize=(1, 153, 222), meta=np.ndarray>
    smad         (time, y, x) float32 136kB dask.array<chunksize=(1, 153, 222), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 68kB dask.array<ch

In [10]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44879 instead
  warnings.warn(


In [11]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [12]:
# Explore the site we are working on
scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [13]:
scaled

<xarray.Dataset> Size: 4MB
Dimensions:      (y: 153, x: 222)
Coordinates:
  * y            (y) float64 1kB -1.987e+06 -1.987e+06 ... -1.988e+06 -1.988e+06
  * x            (x) float64 2kB 3.017e+06 3.017e+06 ... 3.019e+06 3.019e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (y, x) float64 272kB 0.3805 0.3725 0.3798 ... 0.0117 0.0127
    red          (y, x) float64 272kB 0.0338 0.0366 0.0467 ... 0.0148 0.0156
    blue         (y, x) float64 272kB 0.0227 0.0231 0.0285 ... 0.0395 0.0402
    green        (y, x) float64 272kB 0.047 0.0468 0.053 ... 0.0224 0.0228
    emad         (y, x) float32 136kB 0.1446 0.1356 0.1231 ... 0.04104 0.0422
    smad         (y, x) float32 136kB 4.011e-07 5.133e-07 ... 4.317e-06 4.44e-06
    ...           ...
    swir16       (y, x) float64 272kB 0.1977 0.1956 0.2189 ... 0.0135 0.0136
    swir22       (y, x) float64 272kB 0.1071 0.1063 0.1412 ... 0.0131 0.0131
    coastal      (y, x) float64 272kB 0.0716 0.0679 0.0672 ... 0.0485 0.0485
    rededge1     (y, x) float64 272kB 0.1112 0.1085 0.1389 ... 0.0156 0.0156
    rededge2     (y, x) float64 272kB 0.2952 0.2919 0.2873 ... 0.0139 0.0139
    rededge3     (y, x) float64 272kB 0.3599 0.3567 0.3405 ... 0.0152 0.0153

In [14]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [15]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = scaled.blue.max().values
min = scaled.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


In [16]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [17]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": scaled.y[: -pad],
    "x": scaled.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
combined = scaled.copy()
combined = combined.assign(result_bands)

combined

<xarray.Dataset> Size: 8MB
Dimensions:      (y: 153, x: 222)
Coordinates:
  * y            (y) float64 1kB -1.987e+06 -1.987e+06 ... -1.988e+06 -1.988e+06
  * x            (x) float64 2kB 3.017e+06 3.017e+06 ... 3.019e+06 3.019e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/34)
    nir          (y, x) float64 272kB 0.3805 0.3725 0.3798 ... 0.0117 0.0127
    red          (y, x) float64 272kB 0.0338 0.0366 0.0467 ... 0.0148 0.0156
    blue         (y, x) float64 272kB 0.0227 0.0231 0.0285 ... 0.0395 0.0402
    green        (y, x) float64 272kB 0.047 0.0468 0.053 ... 0.0224 0.0228
    emad         (y, x) float32 136kB 0.1446 0.1356 0.1231 ... 0.04104 0.0422
    smad         (y, x) float32 136kB 4.011e-07 5.133e-07 ... 4.317e-06 4.44e-06
    ...           ...
    homogeneity  (y, x) float32 136kB 0.329 0.3415 0.364 0.3749 ... nan nan nan
    energy       (y, x) float32 136kB 0.1211 0.1215 0.1314 ... nan nan nan
    ASM          (y, x) float32 136kB 0.01466 0.01476 0.01726 ... nan nan nan
    correlation  (y, x) float32 136kB 0.8602 0.878 0.8933 0.9049 ... nan nan nan
    mean         (y, x) float32 136kB 8.104 7.917 7.604 7.194 ... nan nan nan
    entropy      (y, x) float32 136kB 6.263 6.309 6.231 5.974 ... nan nan nan

In [18]:
masked_combined, mask = all_masks(combined, return_mask = True)
mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [19]:
masked_combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [20]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(masked_combined.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    masked_combined.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,swir16,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
index,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1986625.0,3017195.0,3832,2024-01-01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1986625.0,3017195.0,3832,2024-01-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1986635.0,3017205.0,3832,2024-01-01
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1986635.0,3017205.0,3832,2024-01-01
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1986635.0,3017215.0,3832,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1987235.0,3016785.0,3832,2024-01-01
303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1987205.0,3016935.0,3832,2024-01-01
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1987305.0,3017205.0,3832,2024-01-01


In [21]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,cc_id,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
35,4.0,0.0302,0.0618,0.0698,0.0915,0.121265,0.000004,0.000020,0.0603,0.1233,...,0.679984,0.281743,0.079379,0.699486,5.298611,4.170808,-1986645.0,3017385.0,3832,2024-01-01
36,4.0,0.0226,0.0484,0.0574,0.0751,0.076271,0.000002,0.000018,0.0269,0.1225,...,0.712500,0.328378,0.107832,0.724882,5.006945,3.828812,-1986655.0,3017385.0,3832,2024-01-01
38,4.0,0.0321,0.0612,0.0694,0.0894,0.082047,0.000002,0.000018,0.0272,0.1226,...,0.723611,0.345691,0.119502,0.739889,4.965278,3.688068,-1986655.0,3017395.0,3832,2024-01-01
39,4.0,0.0237,0.0534,0.0638,0.0831,0.075229,0.000002,0.000018,0.0256,0.1194,...,0.766667,0.396746,0.157407,0.768953,4.791667,3.457460,-1986665.0,3017395.0,3832,2024-01-01
41,4.0,0.0391,0.0742,0.0823,0.1085,0.082849,0.000002,0.000017,0.0367,0.1193,...,0.845833,0.426617,0.182002,0.762797,4.534722,2.997117,-1986655.0,3017425.0,3832,2024-01-01


In [22]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

39 total columns
columns included Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x',
       'spatial_ref', 'time'],
      dtype='object')


In [23]:
standard_schema = ['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x', 'time',
       'spatial_ref']

In [24]:
training_array=training_array[standard_schema]

In [25]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [26]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/csvs/{version}-training.csv", index=False)

In [27]:
training_array["cc_id"].dtype

dtype('float64')